In [1]:
%cd /Users/dom/Desktop/CS224u/cs224u_crossword

/Users/dom/Desktop/CS224u/cs224u_crossword


In [21]:
%cd decrypt-main/

[Errno 2] No such file or directory: 'decrypt-main/'
/Users/dom/Desktop/CS224u/cs224u_crossword/decrypt-main/seq2seq


In [4]:
%pip install -r requirements.txt

     |████████████████████████████████| 8.5 MB 2.6 MB/s eta 0:00:01
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=b4db1542c09d31208e2dbef7adc8375de256320c3f5beb0285b649b07cd02f7f
  Stored in directory: /Users/dom/Library/Caches/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pushd ./data && unzip "*.json.zip" && popd

Archive:  disjoint.json.zip
  inflating: disjoint.json           

Archive:  naive_random.json.zip
  inflating: naive_random.json       

Archive:  disjoint_word_init.json.zip
  inflating: disjoint_word_init.json  

Archive:  guardian_2020_10_08.json.zip
  inflating: guardian_2020_10_08.json  

4 archives were successfully processed.


In [10]:
sys.path.append('/content/drive/MyDrive/github/cs224u_crossword/decrypt-main/decrypt')

In [17]:
%cd /content/drive/MyDrive/github/cs224u_crossword/decrypt-main/

/content/drive/MyDrive/github/cs224u_crossword/decrypt-main


# Test splits

In [22]:
from decrypt.scrape_parse import (
  load_guardian_splits,               # naive random split
  load_guardian_splits_disjoint,      # answer-disjoint split
  load_guardian_splits_disjoint_hash  # word-initial disjoint split
)
from decrypt.scrape_parse.guardian_load import SplitReturn
"""
each of these methods returns a tuple of `SplitReturn`
- soln to clue map (string to List of clues mapping to that soln): Dict[str, List[BaseClue]
this enables seeing all clues associated with a given answer word
- list of all clues (List[BaseClue])
- Tuple of three lists (the train, val, test splits), each is List[BaseClue]

Note that
load_guardian_splits() will verify that
- total glob length matches the one in paper (ie. number of puzzles downloaded matches)
- total clue set length matches the one in paper (i.e. filtering is the same)
- one of the clues in our train set matches our train set (i.e. a single clue
spot check for randomness)
If you get an assertion error or an exception during load, please file an
issue, since the splits should be identical
Alternatively, if you don't care, you can pass `verify=False` to
`load_guardian_splits`
"""

soln_to_clue_map, all_clues_list, (train, val, test) = load_guardian_splits()

INFO:decrypt.scrape_parse.guardian_load:Loading splits directly from given json files. Using /Users/dom/Desktop/CS224u/cs224u_crossword/decrypt-main/data/naive_random.json
100%|██████████| 142380/142380 [00:00<00:00, 942103.07it/s]
INFO:decrypt.scrape_parse.guardian_load:Counter({1: 118540, 2: 20105, 3: 2929, 4: 686, 5: 112, 6: 8})
INFO:decrypt.scrape_parse.guardian_load:Clue list length matches Decrypting paper expected length
INFO:decrypt.scrape_parse.guardian_load:Got splits of lenghts [85428, 28476, 28476]
INFO:decrypt.scrape_parse.guardian_load:First three clues of train set:
	[CleanGuardianClue(clue='Suffering to grasp edge of plant', lengths=[8], soln='agrimony', soln_with_spaces='agrimony', idx=-1, dataset='', across_or_down='', pos=(0, 0), unique_clue_id='', type='cryptic', number=0, id='', creator='Chifonie', orig_lengths='8', lengths_punctuation=set()), CleanGuardianClue(clue='Honour Ben and Noel with new order', lengths=[7], soln='ennoble', soln_with_spaces='ennoble', idx=-

# Baseline t5


In [23]:
%cd baselines

[Errno 2] No such file or directory: 'baselines'
/Users/dom/Desktop/CS224u/cs224u_crossword/decrypt-main/seq2seq


In [10]:
%brew install enchant
%pip install pyenchant

UsageError: Line magic function `%apt` not found.


In [24]:
%load_ext autoreload
%autoreload 2

from decrypt.scrape_parse import (
    load_guardian_splits,
    load_guardian_splits_disjoint,
    load_guardian_splits_disjoint_hash
)

import os
from decrypt import config
from decrypt.common import validation_tools as vt
from decrypt.common.util_data import clue_list_tuple_to_train_split_json
import logging
logging.getLogger(__name__)


k_json_folder = config.DataDirs.Guardian.json_folder

In [25]:
def make_dataset(split_type: str, overwrite=True):
    assert split_type in ['naive_random', 'naive_disjoint', 'word_init_disjoint']
    if split_type == 'naive_random':
        load_fn = load_guardian_splits
        tgt_dir = config.DataDirs.DataExport.guardian_naive_random_split
    elif split_type == 'naive_disjoint':
        load_fn = load_guardian_splits_disjoint
        tgt_dir = config.DataDirs.DataExport.guardian_naive_disjoint_split
    else:
        load_fn = load_guardian_splits_disjoint_hash
        tgt_dir = config.DataDirs.DataExport.guardian_word_init_disjoint_split

    _, _, (train, val, test) = load_fn(k_json_folder)

    os.makedirs(tgt_dir, exist_ok=True)
    # write the output as json
    try:
        clue_list_tuple_to_train_split_json((train, val, test),
                                            comment=f'Guardian data. Split: {split_type}',
                                            export_dir=tgt_dir,
                                            overwrite=overwrite)
    except FileExistsError:
        logging.warning(f'You have already generated the {split_type} dataset.\n'
                        f'It is located at {tgt_dir}\n'
                        f'To regenerate, pass overwrite=True or delete it\n')


make_dataset('naive_random')
make_dataset('word_init_disjoint')
# you can also make_dataset('naive_disjoint')

INFO:decrypt.scrape_parse.guardian_load:Loading splits directly from given json files. Using /Users/dom/Desktop/CS224u/cs224u_crossword/decrypt-main/data/naive_random.json
100%|██████████| 142380/142380 [00:00<00:00, 1168725.82it/s]
INFO:decrypt.scrape_parse.guardian_load:Counter({1: 118540, 2: 20105, 3: 2929, 4: 686, 5: 112, 6: 8})
INFO:decrypt.scrape_parse.guardian_load:Clue list length matches Decrypting paper expected length
INFO:decrypt.scrape_parse.guardian_load:Got splits of lenghts [85428, 28476, 28476]
INFO:decrypt.scrape_parse.guardian_load:First three clues of train set:
	[CleanGuardianClue(clue='Suffering to grasp edge of plant', lengths=[8], soln='agrimony', soln_with_spaces='agrimony', idx=-1, dataset='', across_or_down='', pos=(0, 0), unique_clue_id='', type='cryptic', number=0, id='', creator='Chifonie', orig_lengths='8', lengths_punctuation=set()), CleanGuardianClue(clue='Honour Ben and Noel with new order', lengths=[7], soln='ennoble', soln_with_spaces='ennoble', idx=

{'idx': -1,
 'input': 'Suffering to grasp edge of plant (8)',
 'target': 'agrimony'}
{'idx': -1,
 'input': 'Honour Ben and Noel with new order (7)',
 'target': 'ennoble'}
{'idx': -1, 'input': 'Bit the royal we love? Cheers! (4)', 'target': 'iota'}



100%|██████████| 142380/142380 [00:00<00:00, 1244415.42it/s]
INFO:decrypt.scrape_parse.guardian_load:Counter({1: 118540, 2: 20105, 3: 2929, 4: 686, 5: 112, 6: 8})
INFO:decrypt.scrape_parse.guardian_load:Clue list length matches Decrypting paper expected length
INFO:decrypt.scrape_parse.guardian_load:Got splits of lenghts [75847, 32628, 33905]
INFO:decrypt.scrape_parse.guardian_load:First three clues of train set:
	[CleanGuardianClue(clue='Sailor boy in his hammock', lengths=[4], soln='abed', soln_with_spaces='abed', idx=-1, dataset='', across_or_down='', pos=(0, 0), unique_clue_id='', type='cryptic', number=0, id='', creator='Rufus', orig_lengths='4', lengths_punctuation=set()), CleanGuardianClue(clue='With a degree, I leave this subject', lengths=[5], soln='maths', soln_with_spaces='maths', idx=-1, dataset='', across_or_down='', pos=(0, 0), unique_clue_id='', type='cryptic', number=0, id='', creator='Rufus', orig_lengths='5', lengths_punctuation=set()), CleanGuardianClue(clue='Burrow 

{'idx': -1, 'input': 'Sailor boy in his hammock (4)', 'target': 'abed'}
{'idx': -1,
 'input': 'With a degree, I leave this subject (5)',
 'target': 'maths'}
{'idx': -1,
 'input': 'Burrow to cure limb and make sure one gets up (3,3,5)',
 'target': 'set the alarm'}



In [ ]:
%pip install transformers==4.4.2
%pip install wandb
%pip install torch==1.7.1
%pip install torchvision==0.8.2

In [1]:
%cd /Users/dom/Desktop/CS224u/cs224u_crossword/decrypt-main/seq2seq

/Users/dom/Desktop/CS224u/cs224u_crossword/decrypt-main/seq2seq


In [15]:
%pip install overrides

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:

import typing
import abc

In [35]:
import wandb

wandb.init(project="baseline", entity="224u-s22-cryptic-crosswords",name="baseline_naive")

In [2]:
!ls

62_requirements.txt README.md           decrypt-main


In [40]:
#/usr/bin/env python3
import wandb

run = wandb.init(project="baseline", entity="224u-s22-cryptic-crosswords",name="baseline_naive")
print(run.name)


baseline_naive


In [8]:
!pip install sentencepiece

In [1]:
from transformers import AlbertTokenizer

/opt/anaconda3/envs/224ufinal/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

input_ids = tokenizer("The <extra_id_0> walks in <extra_id_1> park", return_tensors="pt").input_ids
labels = tokenizer("<extra_id_0> cute dog <extra_id_1> the <extra_id_2>", return_tensors="pt").input_ids

# the forward function automatically creates the correct decoder_input_ids
loss = model(input_ids=input_ids, labels=labels).loss
loss.item()

3.7837321758270264

In [2]:
!python train_clues.py --model_name="t5-small" --default_train=base --name=baseline_naive --project=baseline --wandb_dir='./wandb' --data_dir='../data/clue_json/guardian/naive_random'

wandb: Currently logged in as: dterr (224u-s22-cryptic-crosswords). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.12.17
wandb: Run data is saved locally in ./wandb/wandb/run-20220603_154157-2kv6fa2e
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run baseline_naive
wandb: ⭐️ View project at https://wandb.ai/224u-s22-cryptic-crosswords/baseline
wandb: 🚀 View run at https://wandb.ai/224u-s22-cryptic-crosswords/baseline/runs/2kv6fa2e
..... CLEARING
[06.03 15:42:03] [train_clues.py:132 - <module>()]	 train_clues.py --model_name=t5-small --default_train=base --name=baseline_naive --project=baseline --wandb_dir=./wandb --data_dir=../data/clue_json/guardian/naive_random
[06.03 15:42:03] [util.py:160 - set_seed()]	 Setting seed
[06.03 15:42:03] [util_checkpoint.py:65 - __init__()]	 Saver will track (metric, maximize?)
 [('dev/num_match_top_sampled', True), ('epoch', True)]
[06.03 15:42:03] [util.py:78 - get_available_devices()]	 Device: cp

Curricular training
At this point you should have a files at

./data/clue_json/curricular/ACW/train.json
./data/clue_json/curricular/anagram/[train.json, anag_indics.json]
Running curricular training is the same as running main t5 vanilla train, except that we pass an extra multitask flag, which specifies the curriculum to use. See seq2seq/multitask_config. You should pass one of the names from multi_config dict in that file

For example, to train the naive split with the top performing curricular approach (i.e. the result in table 3 that is ACW + ACW-descramble)

In [44]:
!python train_clues.py --default_train=base --name=naive_top_curricular --project=curricular --wandb_dir='./wandb' --data_dir='../data/clue_json/guardian/naive_random' --multitask=ACW__ACW_descramble

wandb: Currently logged in as: dterr (224u-s22-cryptic-crosswords). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.12.17
wandb: Run data is saved locally in ./wandb/wandb/run-20220602_163312-1iy68p1t
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run naive_top_curricular
wandb: ⭐️ View project at https://wandb.ai/224u-s22-cryptic-crosswords/curricular
wandb: 🚀 View run at https://wandb.ai/224u-s22-cryptic-crosswords/curricular/runs/1iy68p1t
..... CLEARING
[06.02 16:33:17] [train_clues.py:132 - <module>()]	 train_clues.py --default_train=base --name=naive_top_curricular --project=curricular --wandb_dir=./wandb --data_dir=../data/clue_json/guardian/naive_random --multitask=ACW__ACW_descramble
[06.02 16:33:17] [util.py:160 - set_seed()]	 Setting seed
[06.02 16:33:17] [util_checkpoint.py:65 - __init__()]	 Saver will track (metric, maximize?)
 [('dev/num_match_top_sampled', True), ('multisave', True), ('multi/acw/num_match_in_sample', T